# Lab 4
### Context

#### AutoML & Experiment Tools
- WanDB

In [ ]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 97 kB 6.5 MB/s 
     |████████████████████████████████| 139 kB 29.2 MB/s 
     |████████████████████████████████| 180 kB 45.3 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=d08713bc5c0f7959c967486a4de284d3988c558a674c5a5951ef3fa6df5748c6
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=cb393e55a910f807268c92fe1bf26047f63edf0f796ac90ca17dac7e9a1651c5
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [ ]:
sweep_config = {
  "name" : "sweep_demo",
  "method" : "bayes",
  "parameters" : {
    "max_depth" : {
      "distribution": "int_uniform",
      "min":2,
      "max":15
    },
    "subsample" :{
      "distribution": "uniform",
      "min": 0.5,
      "max": 1.0
    },
    "colsample_bytree":{
      "distribution": "uniform",
      "min": 0.5,
      "max": 1.0
    }
  },
  "metric":{
      "name": "cv_loss",
      "goal": "minimize"
  }
}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import wandb

wandb.login()

wandb: Currently logged in as: kbjung (use `wandb login --relogin` to force relogin)


True

In [ ]:
import os
from os.path import join

import multiprocessing
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import wandb

sweep_id = wandb.sweep(sweep_config, 
                       project="wandb demo")

n_cpus = multiprocessing.cpu_count()

Create sweep with ID: pdcwdpsw
Sweep URL: https://wandb.ai/kbjung/wandb%20demo/sweeps/pdcwdpsw


In [ ]:
BASE_DIR = '/content/drive/MyDrive/[2021_11]_멋쟁이사자처럼'

train_path = join(BASE_DIR, 'data', 'MDC14', 'train.csv')
test_path  = join(BASE_DIR, 'data', 'MDC14', 'test.csv')

data = pd.read_csv(train_path)
test = pd.read_csv(test_path)

label = data['credit']

In [ ]:
# 불필요한 컬럼 제거
data.drop(columns=['index', 'credit'], inplace=True)
test.drop(columns=['index'],         inplace=True)

In [ ]:
cat_columns = [c for c, t in zip(data.dtypes.index, data.dtypes) if t == 'O'] 
num_columns = [c for c    in data.columns if c not in cat_columns]

print('Categorical Columns: \n{}\n'.format(cat_columns))
print('Numeric Columns: \n{}'.format(num_columns))

Categorical Columns: 
['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type']

Numeric Columns: 
['child_num', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_MOBIL', 'work_phone', 'phone', 'email', 'family_size', 'begin_month']


#### 라벨 데이터 인코딩

In [ ]:
label = label.astype(int)

#### 전처리 프로세스 함수로 작성

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def preprocess(x_train, x_valid, x_test):
    tmp_x_train = x_train.copy()
    tmp_x_valid = x_valid.copy()
    tmp_x_test  = x_test.copy()
    
    tmp_x_train.reset_index(drop=True, inplace=True)
    tmp_x_valid.reset_index(drop=True, inplace=True)
    
    # 결측치 처리
    imputer = SimpleImputer(strategy='most_frequent')
    tmp_x_train[cat_columns] = imputer.fit_transform(tmp_x_train[cat_columns])
    tmp_x_valid[cat_columns] = imputer.transform(tmp_x_valid[cat_columns])
    tmp_x_test[cat_columns]  = imputer.transform(tmp_x_test[cat_columns])
    
    # 스케일링
    scaler = StandardScaler()
    tmp_x_train[num_columns] = scaler.fit_transform(tmp_x_train[num_columns])
    tmp_x_valid[num_columns] = scaler.transform(tmp_x_valid[num_columns])
    tmp_x_test[num_columns]  = scaler.transform(tmp_x_test[num_columns])

    # 인코딩
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(tmp_x_train[cat_columns])
    
    tmp_x_train_cat = pd.DataFrame(ohe.transform(tmp_x_train[cat_columns]))
    tmp_x_valid_cat = pd.DataFrame(ohe.transform(tmp_x_valid[cat_columns]))
    tmp_x_test_cat  = pd.DataFrame(ohe.transform(tmp_x_test[cat_columns]))
    
    tmp_x_train.drop(columns=cat_columns, inplace=True)
    tmp_x_valid.drop(columns=cat_columns, inplace=True)
    tmp_x_test.drop(columns=cat_columns, inplace=True)
    
    tmp_x_train = pd.concat([tmp_x_train, tmp_x_train_cat], axis=1)
    tmp_x_valid = pd.concat([tmp_x_valid, tmp_x_valid_cat], axis=1)
    tmp_x_test  = pd.concat([tmp_x_test, tmp_x_test_cat], axis=1)
    
    return tmp_x_train, tmp_x_valid, tmp_x_test

### Out-of-fold(OOF) Ensemble

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from xgboost import XGBClassifier

def train():
    with wandb.init() as run:
        params = wandb.config
        
        val_scores = list()
        oof_pred = np.zeros((test.shape[0], 3))
        n_splits = 5

        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

        for i, (trn_idx, val_idx) in enumerate(skf.split(data, label)):
            x_train, y_train = data.iloc[trn_idx, :], label.iloc[trn_idx,]
            x_valid, y_valid = data.iloc[val_idx, :], label.iloc[val_idx,]

            # 전처리
            x_train, x_valid, x_test = preprocess(x_train, x_valid, test)

            # 모델 정의
            model = XGBClassifier(n_estimators=1000,
                                  max_depth=params['max_depth'],
                                  subsample=params['subsample'],
                                  colsample_bytree=params['colsample_bytree'],
                                  tree_method='gpu_hist',
                                  n_jobs=n_cpus-1)

            # 모델 학습
            model.fit(x_train, y_train, 
                      eval_metric='mlogloss', 
                      eval_set=[[x_train, y_train], [x_valid, y_valid]],
                      early_stopping_rounds=100,
                      verbose=100)

            # 훈련, 검증 데이터 log_loss 확인
            trn_logloss = log_loss(y_train, model.predict_proba(x_train))
            val_logloss = log_loss(y_valid, model.predict_proba(x_valid))
            print('{} Fold, train logloss : {:.4f}4, validation logloss : {:.4f}'.format(i, trn_logloss, val_logloss))

            val_scores.append(val_logloss)
            
        metrics = {"cv_loss": np.mean(val_scores)}
        wandb.log(metrics)
count = 5
wandb.agent(sweep_id, function=train, count=count)

wandb: Agent Starting Run: 3g4n5mjv with config:
wandb: 	colsample_bytree: 0.5127808294962001
wandb: 	max_depth: 4
wandb: 	subsample: 0.5190283591624516


[0]	validation_0-mlogloss:1.06137	validation_1-mlogloss:1.06136
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 100 rounds.
[100]	validation_0-mlogloss:0.763067	validation_1-mlogloss:0.788083
[200]	validation_0-mlogloss:0.727081	validation_1-mlogloss:0.77684
[300]	validation_0-mlogloss:0.698826	validation_1-mlogloss:0.766506
[400]	validation_0-mlogloss:0.673305	validation_1-mlogloss:0.75988
[500]	validation_0-mlogloss:0.652057	validation_1-mlogloss:0.754742
[600]	validation_0-mlogloss:0.632291	validation_1-mlogloss:0.751188
[700]	validation_0-mlogloss:0.615317	validation_1-mlogloss:0.749252
[800]	validation_0-mlogloss:0.599775	validation_1-mlogloss:0.747752
[900]	validation_0-mlogloss:0.58595	validation_1-mlogloss:0.744756
[999]	validation_0-mlogloss:0.572835	validation_1-mlogloss:0.743786
0 Fold, train logloss : 0.57514, validation logloss : 0.7433
[0]	validation_0-mlogloss:1.06

cv_loss,▁
cv_loss,0.74941


wandb: Agent Starting Run: 9xcasjck with config:
wandb: 	colsample_bytree: 0.885795799067931
wandb: 	max_depth: 13
wandb: 	subsample: 0.6963519089174012


[0]	validation_0-mlogloss:1.04309	validation_1-mlogloss:1.05225
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 100 rounds.
[100]	validation_0-mlogloss:0.315057	validation_1-mlogloss:0.7211
Stopping. Best iteration:
[71]	validation_0-mlogloss:0.390235	validation_1-mlogloss:0.715562

0 Fold, train logloss : 0.39024, validation logloss : 0.7156
[0]	validation_0-mlogloss:1.04495	validation_1-mlogloss:1.05269
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 100 rounds.
[100]	validation_0-mlogloss:0.30253	validation_1-mlogloss:0.741904
Stopping. Best iteration:
[68]	validation_0-mlogloss:0.38798	validation_1-mlogloss:0.731792

1 Fold, train logloss : 0.38804, validation logloss : 0.7318
[0]	validation_0-mlogloss:1.04478	validation_1-mlogloss:1.05334
Multiple eval metrics have bee

...

[0]	validation_0-mlogloss:1.05988	validation_1-mlogloss:1.06078
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 100 rounds.
[100]	validation_0-mlogloss:0.698208	validation_1-mlogloss:0.767069
[200]	validation_0-mlogloss:0.623515	validation_1-mlogloss:0.743366
[300]	validation_0-mlogloss:0.566346	validation_1-mlogloss:0.73117
[400]	validation_0-mlogloss:0.519663	validation_1-mlogloss:0.724741
[500]	validation_0-mlogloss:0.480458	validation_1-mlogloss:0.721177
[600]	validation_0-mlogloss:0.446882	validation_1-mlogloss:0.720079
Stopping. Best iteration:
[594]	validation_0-mlogloss:0.448538	validation_1-mlogloss:0.719865

0 Fold, train logloss : 0.44854, validation logloss : 0.7199
[0]	validation_0-mlogloss:1.05968	validation_1-mlogloss:1.06101
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn

cv_loss,▁
cv_loss,0.72874


wandb: Agent Starting Run: uq5g96kb with config:
wandb: 	colsample_bytree: 0.7215932742634054
wandb: 	max_depth: 8
wandb: 	subsample: 0.5383322511725925


[0]	validation_0-mlogloss:1.05259	validation_1-mlogloss:1.05455
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 100 rounds.
[100]	validation_0-mlogloss:0.59706	validation_1-mlogloss:0.742415
[200]	validation_0-mlogloss:0.468348	validation_1-mlogloss:0.724136
[300]	validation_0-mlogloss:0.386493	validation_1-mlogloss:0.723758
Stopping. Best iteration:
[243]	validation_0-mlogloss:0.429413	validation_1-mlogloss:0.722081

0 Fold, train logloss : 0.42944, validation logloss : 0.7221
[0]	validation_0-mlogloss:1.05176	validation_1-mlogloss:1.05503
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 100 rounds.
[100]	validation_0-mlogloss:0.583902	validation_1-mlogloss:0.749636
[200]	validation_0-mlogloss:0.463116	validation_1-mlogloss:0.734156
[300]	validation_0-mlogloss:0.381352	vali

cv_loss,▁
cv_loss,0.72846
